In [1]:
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json
import shutil

In [2]:
SEED = 178 
WORLD_NAME = 'segmentation_world'
MODEL_NAME_PREFIX = 'tomato'
OUT_PATH = Path(Path.cwd() / '../generated/test_05').resolve()
MODEL_TEMPLATE = Path(Path.cwd() / '../templates/tomato_model').resolve()
WORLD_TEMPLATE = Path(Path.cwd() / '../templates/tomato_world').resolve()
ROW_COUNT = 3
ROW_LENGTH = 20
ROW_DIST = 2.5
CROP_DIST = 0.3

shutil.rmtree(OUT_PATH, ignore_errors=True)
np.random.seed(SEED)


In [3]:
# helper class to build the markers.json
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [4]:
ROW_COUNT *= 2
LL = ROW_LENGTH * CROP_DIST
print(ROW_COUNT, LL)

6 6.0


In [5]:
models = {'list': []}
Markers.reset()

for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = 'tomato_{}'.format(x * ROW_LENGTH + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})
        if x%2 ==0:
            x_pos, y_pos, z_pos = x * (ROW_DIST + CROP_DIST)*0.5, y * CROP_DIST, 0
        else:
            x_pos, y_pos, z_pos = (x-1) * (ROW_DIST + CROP_DIST)*0.5 + CROP_DIST, y * CROP_DIST, 0

        
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        x_pos += np.random.uniform(-0.1, 0.1)
        y_pos += np.random.uniform(-0.1, 0.1)
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )
            

cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use

Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Collada export to: /

Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Collada export to: /home/angel/tdg_ws/src/world_gen/generated/test_05/segmentation_world/tomato_12/meshes/tomato.dae
Info: Exported 8 Objects
Info: Exported 8 Objects


Collada export to: /home/angel/tdg_ws/src/world_gen/generated/test_05/segmentation_world/tomato_18/meshes/tomato.dae
Info: Exported 6 Objects
Info: Exported 6 Objects

Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001198 MB
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use de

Info: Exported 8 Objects
Info: Exported 8 Objects

Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001198 MB
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: 

...

Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Collada export to: /home/angel/tdg_ws/src/world_gen/generated/test_05/segmentation_world/tomato_93/meshes/tomato.dae
Info: Exported 6 Objects
Info: Exported 6 Objects


Collada export to: /home/angel/tdg_ws/src/world_gen/generated/test_05/segmentation_world/tomato_99/meshes/tomato.dae
Info: Exported 10 Objects
Info: Exported 10 Objects

Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001198 MB
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use 


Blender quit
Error: Not freed memory blocks: 7, total unfreed memory 0.001198 MB
Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use d

Color management: using fallback mode for management
Color management: Error could not find role data role.
Blender 3.0.1
Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use

Color management: scene view "Filmic" not found, setting default "Standard".
Read blend: /home/angel/tdg_ws/src/world_gen/blender/tomato_gen.blend
Color management: scene view "Filmic" not found, setting default "Standard".
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Color management: image colorspace "Non-Color" not found, will use default instead.
Collada export to: /